# NLP Assignment 2
*Alessandro Lombardini*  
*Giacomo Melacini*    
*Matteo Rossi Reich*  
*Lorenzo Tribuiani*    

The main aim of this project is to create a neural network based on the models **distilroberta-base** and **bertiny** the deploy a task of *conversational question answering* based on the CoQa dataset.

The basilar idea is to deploy two different models which than will be connected to act like a unique network. The two networks are:

1. **A span extraction model**: The main aim of this network is to extract the span indicies of the rationale inside the context of the given qiestion. By itself the rationale is a valid answer to the given question, but, in order to riformulate it we use a second network

2. **Sequence to Sequence model**: this model, called alse *Encoder Decoder Model* is a particular model thougth to deploy text generation/summarization task. In this specific case will take as inputs the *question* and the *rational* with the specific aim of producing the given **answer**

# General Import & Global Variables


In [ ]:
%%capture
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install dill==0.3.5.1
!pip install --upgrade urllib3

In [ ]:
import os
import urllib.request
from tqdm.notebook import tqdm
import pandas as pd
import json
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import torch
import datasets
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForQuestionAnswering, EncoderDecoderModel
import collections
from transformers.utils import logging
import re 
import string
from tabulate import tabulate
import pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.environ["WANDB_DISABLED"] = "true" # disable weights and biases module auto imported by huggingface

In [ ]:
datasets.disable_progress_bar()       # disable progess bar for output size reason

In [ ]:
# All the model have been stored on a local drive, set False mount drive to work locally
MOUNT_DRIVE = True
OVERRIDE = False
TRAIN_VAL_SEED = 42
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
MODEL_NAMES = ["distilroberta-base", "prajjwal1/bert-tiny"]
SEEDS = [42, 2022, 1337]

In [ ]:
if MOUNT_DRIVE:
  drive.mount('/content/drive')
  SAVING_PATH = os.path.join(os.path.abspath(os.getcwd()), "drive", "MyDrive", "NLP_Assignment_2_spex")
else:
  SAVING_PATH = os.path.join('.', "Savings") 

SAVING_PATH = SAVING_PATH + '/'

if not os.path.exists(SAVING_PATH):
  os.mkdir(SAVING_PATH)

print(SAVING_PATH)

Mounted at /content/drive
/content/drive/MyDrive/NLP_Assignment_2_spex/


In [ ]:
# The f1 score function imported from allennlp_models

def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

def white_space_fix(text):
    return " ".join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def normalize_with_articles(s):
  return white_space_fix(remove_punc(lower(s)))

def get_tokens(s):
    if not s:
        return []
    return normalize_with_articles(s).split()

def compute_exact(a_pred: str, a_gold: str) -> int:
    return int(normalize_answer(a_pred) == normalize_answer(a_gold))

def compute_f1(a_pred: str, a_gold: str) -> float:
    pred_toks = get_tokens(a_pred)
    gold_toks = get_tokens(a_gold)
    common = collections.Counter(pred_toks) & collections.Counter(gold_toks)  # type: ignore[var-annotated]
    num_same = sum(common.values())
    if len(pred_toks) == 0 or len(gold_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return float(pred_toks == gold_toks)
    if num_same == 0:
        return 0.0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [ ]:
# set the wanted seed on the hole environment
def set_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        print("Download completed!\n")

In [ ]:
# function for the description of the percentiles length of the datasets inputs
def describe(dataset, tokenizer, has_labels=False): 

  lengths_inputs = [len(x) for x in dataset["input_ids"]]
  max_input_len = np.max(lengths_inputs)
  input_percentiles = np.percentile(lengths_inputs, q=[50, 75, 85, 90, 95, 99.5], axis=0)
  indexes = ["max length", "50 percentile", "75 percentile", "85 percentile", "90 percentile", "95 percentile", "99.5 percentile"]
  
  counts = [""]
  for percentile in input_percentiles:
    counts.append(np.sum(lengths_inputs > percentile))

  df = pd.DataFrame.from_dict({
      "" : indexes,
      "input ids" : np.insert(input_percentiles, 0, max_input_len),
      "inputs greater" : counts
  })

  if has_labels:
    lengths_labels = [len(x) for x in dataset["labels"]]
    max_label_len = np.max(lengths_labels)
    label_percentiles = np.percentile(lengths_labels, q=[50, 75, 85, 90, 95, 99.5], axis=0)
    df["labels"] = np.insert(label_percentiles, 0, max_label_len)  
    counts = [""]
    for percentile in label_percentiles:
      counts.append(np.sum(lengths_labels > percentile))
    df["label greater"] = counts

  df.set_index("", inplace=True)

  return df

# Data Parsing and Analysis

The first step is to analyze the Dataset. As defined by the assigment the CoQa dataset is used. The datasets contains a list of context (the stories to which the questions are about), a series of questions and answer. Moreover, also a list of rationales and span indicies are given. The dataset is loaded and parsed into Pandas dataframe with the following columns:

- **id**: a unique id reference used for the sliding window
- **question**: the given question about the context
- **answer**: the answer to the given question
- **context**: the context to which the question is about
- **rationale**: A subsequence of the context in ehich the answer to the given question is expressed. This field (togheter with the *spen end* and the *span start*) will be used for the training of the span extraction model
- **span start**: the index of the starting token of the rationale inside the context
- **span end**: the index of the final token of the rationale inside the context
- **history**: this is a list of the previous question made about the same context. Thi field will be used inside the model to extract information about question whose answer is referred to previous once 
- **source**: the source of the text (only in test dataset for error nalysis)

In [ ]:
class CoQaParser:
  """
  Class for Loading and parsing the dataset
  Parsed dataset will be stored at the path specified by SAVING_PATH, if you want to store locally set MOUNT_DRIVE = False
  """
  def __load_and_parse(self):
    """
    Load CoQa dataset and parse it creating a specific field history
    """
    json_data_train = json.load(open(f'coqa/train.json'))
    json_data_test = json.load(open(f'coqa/test.json'))

    qa_train = []
    qa_test = []
    # Convert the train dataset
    for data in json_data_train["data"]:
      
      for j in range(0, len(data["questions"])):
        qa_dict = {
            "question" : data["questions"][j]["input_text"],
            "answer" : data["answers"][j]["input_text"],
            "rationale" : data["answers"][j]["span_text"],
            "span_start" : data["answers"][j]["span_start"], 
            "span_end" : data["answers"][j]["span_end"],
            "context" : data["story"]      
        }
        
        history = ""
        for i in range(j):
          history += data["questions"][i]["input_text"] + data["answers"][i]["input_text"] + "[SEP]"
        
        qa_dict["history"] = history
        qa_train.append(qa_dict) 

    # Convert the test dataset
    for  data in json_data_test["data"]:
      for j in range(0, len(data["questions"])):
        qa_dict = {
            "question" : data["questions"][j]["input_text"],
            "answer" : data["answers"][j]["input_text"],
            "rationale" : data["answers"][j]["span_text"],
            "span_start" : data["answers"][j]["span_start"], 
            "span_end" : data["answers"][j]["span_end"],
            "context" : data["story"],
            "source" : data["source"]                                 # for the test set the source element is imported to (not necessary for train/val)    
        }

        history = ""
        for i in range(j):
          history += data["questions"][i]["input_text"] + data["answers"][i]["input_text"] + "[SEP]"
        
        qa_dict["history"] = history
        qa_test.append(qa_dict)

    qa_train = pd.DataFrame(qa_train)
    qa_test = pd.DataFrame(qa_test)

    return qa_train, qa_test

  def load(self):
    """
    Load CoQa dataset parsed if they are stored into the saving path, otherwise it downloads it and parse it
    """
    
    if (os.path.exists(os.path.join(SAVING_PATH, 'qa_train.pkl')) and
        os.path.exists(os.path.join(SAVING_PATH, 'qa_test.pkl'))):
      
        QA_df_train = pd.read_pickle(os.path.join(SAVING_PATH, 'qa_train.pkl'))
        QA_df_test = pd.read_pickle(os.path.join(SAVING_PATH, 'qa_test.pkl'))
    else:
     
        if not os.path.exists('coqa/train.json'):
           url = f"https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
           download_data(data_path='coqa', url_path=url, suffix="train")
        if not os.path.exists('coqa/test.json'):
           url = f"https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
           download_data(data_path='coqa', url_path=url, suffix="test")
           
        QA_df_train, QA_df_test = self.__load_and_parse()
        QA_df_train.to_pickle(os.path.join(SAVING_PATH, 'qa_train.pkl'))
        QA_df_test.to_pickle(os.path.join(SAVING_PATH, 'qa_test.pkl'))
    
    return QA_df_train, QA_df_test

In [ ]:
qa_train, qa_test =  CoQaParser().load()

One of the most important things to do is to remove the *unknown* answers. Since the problem is not dealing with them all the answers marked as **unknown** will be removed

In [ ]:
def clean_dataframe(dataframe):
  original_size = dataframe.shape
  dataframe = dataframe[dataframe["answer"] != "unknown"]
  dataframe.reset_index(drop = True, inplace = True)
  
  print(f"Original dataframe size: {original_size}")
  print(f"Dataframe size after cleaning: {dataframe.shape}\nremoved {original_size[0] - dataframe.shape[0]} unknown answers\n")
  
  return dataframe

In [ ]:
qa_train = clean_dataframe(qa_train)
qa_test = clean_dataframe(qa_test)

Original dataframe size: (108647, 7)
Dataframe size after cleaning: (107276, 7)
removed 1371 unknown answers

Original dataframe size: (7983, 8)
Dataframe size after cleaning: (7917, 8)
removed 66 unknown answers



Since some of the inputs may be greather than the maximum input size of some of the models used an ID column is added. Even tough and id is given in the dataset this one is specific for the group of question (questions about the same context has the same id) a unique id for each question is needed. This id will be used later during the data preparation in order to keep track of segments of the same text that have been truncated to the maximum input size of the model

In [ ]:
qa_test.insert(0, "id", range(0, len(qa_test)))

In [ ]:
qa_train, qa_val = train_test_split(
    qa_train,
    train_size=0.80,
    test_size=0.20,
    random_state = TRAIN_VAL_SEED # 42
)

for compatibility and usage the dataframes are converted into pytorch dataframe

In [ ]:
qa_train = datasets.Dataset.from_pandas(qa_train)
qa_val = datasets.Dataset.from_pandas(qa_val)
qa_test = datasets.Dataset.from_pandas(qa_test)

## Analysis of inputs length

All the model that are going to be used in this project has standard limit of length for inputs. For the span extraction model a sliding window is implemented. This leads to no unparsed or unseen text, but the sequence to sequence model won't have this window

In [ ]:
def unpadded_tokenization(examples, tokenizer, history=False):
  context = examples["context"] if not history else [hist + cont
                                                     for cont, hist
                                                     in zip(examples["context"], examples["history"])]

  tokenized_inputs = tokenizer(
      examples["question"],
      context,
      padding=False
  )  
  return {"input_ids" : tokenized_inputs["input_ids"]}

In [ ]:
def unpadded_tokenization_s2s(examples, tokenizer):
  tokenized_inputs = tokenizer(
      examples["question"],
      examples["rationale"],
      padding=False
  )
  tokenized_outputs = tokenizer(
      examples["answer"],
      padding=False
  )

  return {
      "input_ids" : tokenized_inputs["input_ids"],
      "labels" : tokenized_outputs["input_ids"]
  }

### Span Extraction

#### No History

In [ ]:
dataframes = {}
for model_name in MODEL_NAMES: 
  logging.set_verbosity_error() # suppress unwanted output logging
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenized_train = qa_train.map(
      unpadded_tokenization,
      fn_kwargs = {"tokenizer" : tokenizer},
      batched=True
  )
  logging.set_verbosity_info() # restore logging level
  dataframes[model_name] = describe(tokenized_train, tokenizer)  

In [ ]:
dataframes[MODEL_NAMES[0]]

,input ids,inputs greater
,,
max length,1392.0,
50 percentile,375.0,42368
75 percentile,412.0,21135
85 percentile,434.0,12840
90 percentile,452.0,8462
95 percentile,485.0,4262
99.5 percentile,806.0,422


In [ ]:
dataframes[MODEL_NAMES[1]]

,input ids,inputs greater
,,
max length,1338.0,
50 percentile,359.0,42452
75 percentile,396.0,21339
85 percentile,419.0,12776
90 percentile,438.0,8432
95 percentile,468.0,4287
99.5 percentile,793.0,429


As can be seen in the tables above both of the model has a maximum input size much larger tha the model expected input size (512). Since a sliding window is developed in the tokenization system there's no need to maximize the input size, but still a reasonable reduction is preferred since reducing the input size too much would result into a huge fragmentation of the dataset itself, the following lengths are chosen for the distilroberta and bertiny models

In [ ]:
SPEX_LEN = {
    MODEL_NAMES[0] : {
        "input_len" : 412
    },
    MODEL_NAMES[1] : {
        "input_len" : 396
    }
}

#### History

In [ ]:
dataframes = {}
for model_name in MODEL_NAMES: 
  logging.set_verbosity_error() # suppress unwanted output logging
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenized_train = qa_train.map(
      unpadded_tokenization,
      fn_kwargs = {"tokenizer" : tokenizer, "history": True},
      batched=True
  )
  logging.set_verbosity_info() # restore logging level
  dataframes[model_name] = describe(tokenized_train, tokenizer) 

In [ ]:
dataframes[MODEL_NAMES[0]]

,input ids,inputs greater
,,
max length,1612.0,
50 percentile,476.0,42655
75 percentile,553.0,21252
85 percentile,594.0,12836
90 percentile,623.0,8484
95 percentile,666.0,4243
99.5 percentile,927.0,429


In [ ]:
dataframes[MODEL_NAMES[1]]

,input ids,inputs greater
,,
max length,1597.0,
50 percentile,439.0,42843
75 percentile,503.0,21405
85 percentile,539.0,12825
90 percentile,564.0,8550
95 percentile,606.0,4229
99.5 percentile,879.0,426


As expected, the input with the history of the previous couples Answer-Question are much bigger. For computational reasons (mainly due to training time that would become exessively long for bigger input sizes) the lengths are not increased, but the previous once are kept 

### Seq2Seq

In [ ]:
dataframes = {}
for model_name in MODEL_NAMES: 
  logging.set_verbosity_error() # suppress unwanted output logging
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenized_train = qa_train.map(
      unpadded_tokenization_s2s,
      fn_kwargs = {"tokenizer" : tokenizer},
      batched=True
  )
  logging.set_verbosity_info() # restore logging level
  dataframes[model_name] = describe(tokenized_train, tokenizer, has_labels=True) 


In [ ]:
dataframes[MODEL_NAMES[0]]

,input ids,inputs greater,labels,label greater
,,,,
max length,470.0,,141.0,
50 percentile,20.0,40551,5.0,31311
75 percentile,27.0,19984,7.0,15094
85 percentile,32.0,11950,8.0,10787
90 percentile,36.0,8021,9.0,7834
95 percentile,43.0,4173,12.0,3360
99.5 percentile,76.0,423,22.0,400


In [ ]:
dataframes[MODEL_NAMES[1]]

,input ids,inputs greater,labels,label greater
,,,,
max length,447.0,,149.0,
50 percentile,19.0,39290,4.0,39812
75 percentile,26.0,19361,6.0,19657
85 percentile,30.0,12865,8.0,9986
90 percentile,35.0,7791,9.0,7391
95 percentile,42.0,4062,11.0,4223
99.5 percentile,74.0,428,22.0,401


As can be seen in the table above there's a large diversity over the input lengths, but still, for inputs and for labels, the great majority of samples has length minor than 74 for inputs and 22 for outputs. However, since those length are small, is possible to thing about a standard size and see how many features will be truncated

In [ ]:
over128 = tokenized_train.filter(lambda x: len(x["input_ids"]) > 128)
print(f"inputs in train dataset longer than 128: {len(over128)}")

inputs in train dataset longer than 128: 83


In [ ]:
over128 = tokenized_train.filter(lambda x: len(x["labels"]) > 128)
print(f"labels in train dataset longer than 128: {len(over128)}")

labels in train dataset longer than 128: 1


For input size of **128** we only have 83 elements truncated, which represents less than 0.5% of the entire dataset. Labels are almost entirely included in length of 128 with just one exception of length 149

In [ ]:
SEQ2SEQ_LEN = {
    MODEL_NAMES[0] : {
        "input_len" : 128,
        "output_len" : 128
    },
    MODEL_NAMES[1] : {
        "input_len" : 128,
        "output_len" : 128
    }
}

# Models Definition

## Model: Span Extractor

The first model defined is the **span extraction** model. A class is defined which, based on the model name, load a Huggingface's **ModelForQuestionAnswering**. The class implements the *preprocess* and *train* methods:

- **train_val_preprocess**: This function is responsible for the tokenization and formattion of the inputs. Mainly 2 process are held:
  - *tokenization* process which is held by the tokenizer itself. Since a sliding window is developed the inputs are expected to be separated into smaller chuncks whenever they are too big. The idea is to divide only the context (and eventually the history if needed) creating chunks with the same structure: $question + chunk_i$. 
  - *inputs organization*. Since the inputs are divided into small chunks the other inputs needs to be updated too, in particular the labels. In this specific case the labels are the index of start and end of the rationale. to keep them coherent with the input, if they are divided into $n$ chunks tha the labels will be represented as $n$ tuple containing $(0,0)$ if the rationale is not in that chunk of the context, $(span\_start, span\_end)$ if the rationale is contained. For simplicity the rationales splitted into more than one chunk will be revoved

- **test_preprocess**: This function tokenize the dataset. Is identical to the provious one with the exception that no labels are created

- **generate**: The generate function produce the rationale according to the output produced from the models. Since the output of the model are **logits**, the 20 best score logits are taken (for the start token and end token) and the answer with the highest score (expressed as $start\_logit + end\_logit$). The **offset mapping**, produced by the tokenizer, and the **id** inserted into the dataset are used to map the logits index to the complete context index and each answer to the specific context.

In [ ]:
class SpanExModel(object):

  def __init__(self, model_name=None, device='cuda', max_input_len=512, stride=128):
    self.model_name = model_name
    self.MAX_LEN = max_input_len
    self.STRIDE = stride
    self.device = device
    self.n_best = 20
    if model_name is not None:
      print(f"Loading {self.model_name} model on SpanEx...")
      logging.set_verbosity_error() #Suppress unwant ouput logging
      self.__tokenizer = AutoTokenizer.from_pretrained(self.model_name)
      self.__model = AutoModelForQuestionAnswering.from_pretrained(self.model_name).to(self.device)
      logging.set_verbosity_info() #Restore regular logging
      print("...Loaded!")

  def cpu(self):
    self.__model.cpu()
  
  def save_model(self, folder_path, model_name):
    self.__model.save_pretrained(
        os.path.join(folder_path, model_name)
    )

  def load_model(self, path, tokenizer_name):
    self.__tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    self.__model = AutoModelForQuestionAnswering.from_pretrained(path).to(self.device)
  

  # Train and validation dataset preprocessing function
  def train_val_preprocess(self, examples, history=False):

    questions = [question.strip() for question in examples["question"]]
    context = examples["context"] if not history else [hist + cont        #if history is required it gets loaded from the dataset and added to the context
                                                       for hist, cont
                                                       in zip(examples["history"], examples["context"])]

    inputs = self.__tokenizer(
        questions,
        context,
        max_length=self.MAX_LEN,
        truncation='only_second',
        stride=self.STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length'
    )

    offset_mapping = inputs.pop("offset_mapping")         # the offset mapping is remove and stored
    sample_map = inputs.pop("overflow_to_sample_mapping") # the overflow to sample mapping is removed from the dict (is useless in this case)
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):           # from the offset mapping the index of the rows are collected
      sample_idx = sample_map[i]
      start_char = examples["span_start"][sample_idx]
      end_char = examples["span_end"][sample_idx]
      sequence_ids = inputs.sequence_ids(i)

      # skip the question
      idx = 0
      while sequence_ids[idx] != 1:
        idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
        idx += 1
      context_end = idx - 1

      if offset[context_start][0] > start_char or offset[context_end][1] < end_char: # if the rational is not in the chunk the labels are (0,0)
        start_positions.append(0)
        end_positions.append(0)
      else:                                                                          # otherwise we search tham into the chunk and we update their values
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

  #test preprocessing function
  def test_preprocess(self, examples, history=False):
    questions = [question.strip() for question in examples["question"]]   # remove useless initial white spaces
    context = examples["context"] if not history else [hist + cont        # if history is required it gets loaded from the dataset and added to the context
                                                       for hist, cont
                                                       in zip(examples["history"], examples["context"])]

    inputs = self.__tokenizer(
        questions,
        context,
        max_length=self.MAX_LEN,
        truncation='only_second',
        stride=self.STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length'
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

  def train_model(
      self,
      train_df,
      val_df,
      output_dir="/",
      epochs=3,
      batch_size=8,
      lr=5e-5
      ):

      training_args = TrainingArguments(
          output_dir,
          do_train=True,
          do_eval=True,
          evaluation_strategy="epoch",
          per_device_train_batch_size=batch_size,
          per_device_eval_batch_size=batch_size,
          learning_rate=lr,
          num_train_epochs=epochs,
          save_strategy="no",
          report_to = "none"
      )

      trainer = Trainer(
          self.__model,
          args=training_args,
          train_dataset=train_df,
          eval_dataset=val_df
      )

      trainer.train()

  # Generate the rationale from the generated outputs
  def generate(self, dataset_batch, history=False):
    
    predicted_answers = []
    dataset_batch = datasets.Dataset.from_dict(dataset_batch)

    dataset_batch_mapped = dataset_batch.map(
        self.test_preprocess,
        fn_kwargs={"history" : history},
        batched=True,
        remove_columns = dataset_batch.column_names
    )

    test_set_for_model = dataset_batch_mapped.map(
      remove_columns=["example_id", "offset_mapping"])

    test_set_for_model.set_format(type='torch', columns=["input_ids", "attention_mask"])

    #batch = {k: test_set_for_model[k] for k in test_set_for_model.column_names}

    outputs = self.__model(
        input_ids = test_set_for_model["input_ids"].to(self.device),
        attention_mask = test_set_for_model["attention_mask"].to(self.device)
    )

    with torch.no_grad():
      start_logits = outputs.start_logits.cpu().numpy()
      end_logits = outputs.end_logits.cpu().numpy()  

    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(dataset_batch_mapped):
        example_to_features[feature["example_id"]].append(idx)

    for example in dataset_batch:
      example_id = example["id"]
      context = example["context"]
      answers = []

      for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = dataset_batch_mapped["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -self.n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -self.n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is < 0 .
                if end_index < start_index:
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

      best_answer = max(answers, key=lambda x: x["logit_score"])
      predicted_answers.append(best_answer["text"])
    
    return predicted_answers
  
  def __del__(self):
    del self.__model

## Model: Answer Formatter

The second model used is the **Sequence2Sequence** or **EncoderDecoderModel** from hugging face. This model is used to format the rationale (given by the first model) to the answer wanted from the dataset. Again this model is represented as a class with the following methods:

- **train_val_preprocess**: this method is responsible for the train and validation preprocessing. In this case the process is much simpler, inputs and labels are tokenized according to the tokenizer used and truncated at the specific max length (or padded)

- **generate**: the generate function creates the wanted text from the logits prediction of the model. In this case a basilar greedy search is used

In [ ]:
class Seq2SeqModel(object):

  def __init__(self, model_name=None, encoder_max_len=128, decoder_max_len=128, device='cuda'):
    self.encoder_max_len = encoder_max_len
    self.decoder_max_len = decoder_max_len
    self.device=device
    if model_name is not None:
      self.model_name = model_name
      print(f"Loading {self.model_name} model on Seq2Seq...")
      logging.set_verbosity_error() #Suppress unwanted ouputs logging
      self.__tokenizer = AutoTokenizer.from_pretrained(self.model_name)
      self.__model = EncoderDecoderModel.from_encoder_decoder_pretrained(self.model_name, self.model_name).to(self.device) 
      logging.set_verbosity_info() #Restore regular logging
      print("...Loaded!")    
      self.__model.config.decoder.is_decoder = True
      self.__model.config.decoder.add_cross_attention = True
      self.__model.config.decoder_start_token_id = self.__tokenizer.cls_token_id
      self.__model.config.pad_token_id = self.__tokenizer.pad_token_id
      self.__model.config.vocab_size = self.__model.config.encoder.vocab_size
  
  # freeing space from the gpu
  def cpu(self):
    self.__model.cpu()
  
  def test_preprocess(self, examples):
    question = examples["question"] 
    rationale = examples["rationale"]

    tokenized_inputs = self.__tokenizer(question,
                                        rationale, 
                                        truncation="only_second", 
                                        max_length=self.encoder_max_len, 
                                        padding="max_length")


    examples["input_ids"] = tokenized_inputs.input_ids
    examples["attention_mask"] = tokenized_inputs.attention_mask

    return examples

  def train_val_preprocess(self, examples):
    question = examples["question"] 
    rationale = examples["rationale"]
    answer = examples["answer"]

    tokenized_inputs = self.__tokenizer(question,
                                        rationale, 
                                        truncation="only_second", 
                                        max_length=self.encoder_max_len, 
                                        padding="max_length")

    tokenized_outputs = self.__tokenizer(answer, 
                                  truncation=True, 
                                  max_length=self.decoder_max_len, 
                                  padding="max_length")

    examples["input_ids"] = tokenized_inputs.input_ids
    examples["attention_mask"] = tokenized_inputs.attention_mask
    examples["decoder_input_ids"] = tokenized_outputs.input_ids
    examples["decoder_attention_mask"] = tokenized_outputs.attention_mask
    examples["labels"] = tokenized_outputs.input_ids.copy()


    # We have to make sure that the PAD token is ignored
    examples["labels"] = [[-100 if token == self.__tokenizer.pad_token_id else token for token in labels] for labels in examples["labels"]]

    return examples
  
  # function for computiong f1 score on the validation during training
  def compute_metrics(self, values):
    target, prediction = values                                                        # gets prediction and gold values
    pred_str = self.__tokenizer.batch_decode(prediction, skip_special_tokens=True)    
    target[target == -100] = self.__tokenizer.pad_token_id
    target_str = self.__tokenizer.batch_decode(target, skip_special_tokens=True)

    return np.mean(
        [compute_f1(pred_str[i], target_str[i]) for i in range(len(values[0]))]
    )
  
  def save_model(self, folder_path, model_name):
    self.__model.save_pretrained(
        os.path.join(folder_path, model_name)
    )
  
  def load_model(self, model_path, tokenizer_name):
    self.model_name = tokenizer_name
    self.__tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    self.__model = EncoderDecoderModel.from_pretrained(model_path).to(self.device)
    self.__model.config.decoder.is_decoder = True
    self.__model.config.decoder.add_cross_attention = True
    self.__model.config.decoder_start_token_id = self.__tokenizer.cls_token_id
    self.__model.config.pad_token_id = self.__tokenizer.pad_token_id
    self.__model.config.vocab_size = self.__model.config.encoder.vocab_size

  def train_model(
      self,
      train_df,
      val_df,
      output_dir="/",
      epochs=3,
      batch_size=8,
      lr=5e-5
      ):

      train_loader = torch.utils.data.DataLoader(train_df, batch_size=batch_size, shuffle=True, drop_last=False)
      val_loader = torch.utils.data.DataLoader(val_df, batch_size=batch_size, shuffle=False)

      self.__model.to(self.device) # switch model to device requested

      optimizer = torch.optim.Adam(self.__model.parameters(), lr=lr) # adam optimizer
      
      general_progress_bar = tqdm(range(len(train_loader) * epochs))
      for epoch in range(1, epochs+1):
          
          # Training
          self.__model.train()
          train_loss = 0  # initial loss

          train_progress_bar = tqdm(train_loader)
          train_progress_bar.set_description(f"Epoch {epoch}")

          for batch in train_progress_bar:
              input_ids = batch["input_ids"].to(self.device)
              attention_mask = batch["attention_mask"].to(self.device)
              labels = batch["labels"].to(self.device)

              output = self.__model(input_ids=input_ids, 
                              attention_mask=attention_mask, 
                              labels=labels)

              loss = output.loss     # compute  loss
              optimizer.zero_grad()  # zeros the gradients
              loss.backward()        # gradient descent
              optimizer.step()
              train_loss += loss.item()

              train_progress_bar.set_postfix(loss=loss.item())
              general_progress_bar.update(1)

          avg_train_loss = train_loss / len(train_loader)

          # Validation
          self.__model.eval()
          valid_loss = 0 
          f1 = 0
          validation_progress_bar = tqdm(val_loader, leave=False)
          validation_progress_bar.set_description("Running Validation")

          with torch.no_grad():
              for batch in validation_progress_bar:
                  input_ids = batch["input_ids"].to(self.device)
                  attention_mask = batch["attention_mask"].to(self.device)
                  labels = batch["labels"].to(self.device)

                  output = self.__model(input_ids=input_ids, 
                              attention_mask=attention_mask, 
                              labels=labels)

                  pred_ids = self.__model.generate(input_ids=input_ids, 
                                                attention_mask=attention_mask)

                  loss = output.loss
                  valid_loss += loss.item()
                  batch_f1 = self.compute_metrics((labels, pred_ids)) # compute f1 score
                  f1 += batch_f1

                  validation_progress_bar.set_postfix(loss=loss.item(), f1=batch_f1.item())

              avg_val_loss = valid_loss / len(val_loader)
              avg_f1_score = f1 / len(val_loader)
          
          train_progress_bar.disable = True
          print("-" * 100)
          print(f"End of epoch {epoch}, results: ")
          print(f'train_loss = {avg_train_loss:.2f}\t val_loss = {avg_val_loss:.2f}\t val_f1 = {avg_f1_score:.3f} ({avg_f1_score*100:.1f}%)')
          print("-" * 100)


  def generate(self, inputs):
    generated = self.__model.generate(                           # greedy search generation
        input_ids=inputs["input_ids"].to(self.device),
        attention_mask=inputs["attention_mask"].to(self.device)
    )
    return self.__tokenizer.batch_decode(generated, skip_special_tokens=True)

  def decode(self, input_ids):
    return self.__tokenizer.batch_decode(input_ids)

  def __del__(self):
    del self.__model


# Complete Model

The complete model is created combining the previous model. the span extraction will produce the rationale that will later be passed, togheter with the question, into the second model for producing the complete answer.



In [ ]:
class CoQaModel(object):

  def __init__(self, model_name=None, span_input_len=512, s2s_input_len=128, s2s_output_len=128, device='cuda'):
    self.model_name = model_name
    self.device = device
    
    if model_name is not None:
        #Span extraction model
        self.spex_model = SpanExModel(
            self.model_name, 
            device=device,
            max_input_len=span_input_len)
        #Sequence to sequence model
        self.s2s_model = Seq2SeqModel(
            self.model_name, 
            device=device,
            encoder_max_len=s2s_input_len,
            decoder_max_len=s2s_output_len)
    else:
        self.spex_model = SpanExModel()
        self.s2s_model = Seq2SeqModel()
  
  # freeing space from gpu ram
  def cpu(self):
    self.spex_model.cpu()
    self.s2s_model.cpu()

  # load the complete model from saving (span ex + seq2seq) 
  def load_model(self, path, tokenizer_name):
    self.model_name = tokenizer_name
    logging.set_verbosity_error()    
    self.spex_model.load_model(
        os.path.join(path, "span_extractor"),
         tokenizer_name
    )  
    self.s2s_model.load_model(
        os.path.join(path, "sequence_2_sequence"),
         tokenizer_name
    )
    logging.set_verbosity_info()

  # general training of both model    
  def train_model(
      self,
      train_df,
      val_df,
      output_dir="/",
      epochs=3,
      batch_size=32,
      lr=5e-5,
      seed=42,
      history=False,
      train_from_predicted=False
  ):
    """
    Training of the general model. trains first the span extractor model and the seq2seq model than
    """
    set_seed(seed)
    


    name = 'bert-tiny' if self.model_name == "prajjwal1/bert-tiny" else 'distilroberta-base' \
       + str(seed) \
       + '_' \
       + ('withHistory' if history else 'withoutHistory')
    
    if os.path.exists(output_dir + name):
        print("°" * 100)
        print(f"Already exists {self.model_name} model and seed {seed}")
        print("°" * 100)
        return
    
    print("°" * 100)
    print(f"Starting training with {self.model_name} model and seed {seed}")
    print("°" * 100)
    print("\nTokenizing Dataset for Span Extraction...")
              
    # dataset preprocess 
    spex_train = train_df.map(
        self.spex_model.train_val_preprocess,
        fn_kwargs={"history" : history},
        batched=True,
        remove_columns=train_df.column_names
    )

    spex_val = val_df.map(
        self.spex_model.train_val_preprocess,
        fn_kwargs={"history" : history},
        batched=True,
        remove_columns=val_df.column_names
    )

    spex_train.set_format('torch')
    spex_val.set_format('torch')
    print("Done!")
    print()
    print("°"*100)
    print("Training Span Extraction Model")
    print("°"*100)

    # Training
    self.spex_model.train_model(
        train_df=spex_train,
        val_df=spex_val,
        output_dir=output_dir,
        epochs=epochs,
        batch_size=batch_size,
        lr=lr
    )
    print("\nTokenizing Dataset for Seq2Seq Model")
    
    # preprocessing seq2seq datasets
    s2s_train = train_df.map(
        self.s2s_model.train_val_preprocess,
        batched=True
    )

    s2s_val = val_df.map(
        self.s2s_model.train_val_preprocess,
        batched=True
    )

    s2s_train.set_format('torch')
    s2s_val.set_format('torch')
    
    print("Done")
    print()
    print("°"*100)
    print("Training Seq2Seq Model")
    print("°"*100)
    # training
    self.s2s_model.train_model(
        train_df=s2s_train,
        val_df=s2s_val,
        output_dir=output_dir,
        epochs=epochs,
        batch_size=batch_size,
        lr=lr
    )
    
    # model saving
    name = 'bert-tiny' if self.model_name == "prajjwal1/bert-tiny" else 'distilroberta-base' \
           + str(seed) \
           + '_' \
           + 'withHistory' if history else 'withoutHistory'
    folder_path = os.path.join(output_dir, name)
    self.spex_model.save_model(folder_path, "span_extractor")
    self.s2s_model.save_model(folder_path, "sequence_2_sequence")


  def generate(self, input_batch, history=False):
    rationales = self.spex_model.generate(input_batch, history=False)
    questions = input_batch["question"]
    
    s2s_input = datasets.Dataset.from_dict({
        "rationale" : rationales,
        "question" : questions
    })

    s2s_input = s2s_input.map(
        self.s2s_model.test_preprocess,
        batched=True
    )

    s2s_input.set_format('torch')

    outputs = self.s2s_model.generate(s2s_input)
    return outputs

  def __del__(self):
      del self.spex_model
      del self.s2s_model
      del self


## Model Training 

### No History

In [ ]:
for model_name in MODEL_NAMES:
  for seed in SEEDS:
    model = CoQaModel(
        model_name, 
        span_input_len=SPEX_LEN[model_name]["input_len"],
        s2s_input_len=SEQ2SEQ_LEN[model_name]["input_len"],
        s2s_output_len=SEQ2SEQ_LEN[model_name]["output_len"],
        device=DEVICE
    )
    
    model.train_model(
        qa_train,
        qa_val,
        seed=seed,
        output_dir = SAVING_PATH
    )

### History

In [ ]:
for model_name in MODEL_NAMES:
  for seed in SEEDS:
    model = CoQaModel(
        model_name, 
        span_input_len=SPEX_LEN[model_name]["input_len"],
        s2s_input_len=SEQ2SEQ_LEN[model_name]["input_len"],
        s2s_output_len=SEQ2SEQ_LEN[model_name]["output_len"],
        device=DEVICE
    )
    
    model.train_model(
        qa_train,
        qa_val,
        seed=seed,
        output_dir = SAVING_PATH
        history=True
    )

## Models Evaluation

The following section is dedicated to the models evaluation. Each models predictions will be evaluated on the test and, base on the prediction, the mean f1 score is calculates, the models are:

- *without history*
  - *distilroberta-base 42*
  - *distilroberta-base 2022*
  - *distilroberta-base 1337*
  - *bertiny 42*
  - *bertiny 2022*
  - *bertiny 1337*

- *with history*
  - *distilroberta-base 42*
  - *distilroberta-base 2022*
  - *distilroberta-base 1337*
  - *bertiny 42*
  - *bertiny 2022*
  - *bertiny 1337*

In [ ]:
def model_evaluation(model, test, batch_size=5):
    iteration_steps = len(test) // batch_size

    batch_start = 0
    batch_end = batch_size

    test_f1 = 0
    
    print("°"*100)
    print("Computing f1 score")
    print("°"*100)
    
    progress_bar = tqdm(range(iteration_steps))
    while batch_end <= len(test):  
      batch = test[batch_start : batch_end]
      logging.set_verbosity_error()
      predicted = (model.generate(batch))
      logging.set_verbosity_info() 

      batch_f1 = 0
      for i, answer in enumerate(predicted):
        batch_f1 += compute_f1(answer, batch["answer"][i])

      batch_f1 /= batch_size
      test_f1 += batch_f1

      batch_start = batch_end 
      batch_end += batch_size

      progress_bar.update(1)

      if batch_end >= len(test):
        batch_end = len(test)

      if batch_start == batch_end:
        batch_end = len(test) + 1

    batch_start = 0
    batch_end = batch_size

    return test_f1/iteration_steps


In [ ]:
models = {
    MODEL_NAMES[0] : {
        False : ["distilroberta-base42_withoutHistory", "distilroberta-base2022_withoutHistory", "distilroberta-base1337_withoutHistory"],
        True : ["distilroberta-base42_withHistory", "distilroberta-base2022_withHistory", "distilroberta-base1337_withHistory"]
    },
    MODEL_NAMES[1] : {
        False : ["ber-tiny42_withoutHistory", "ber-tiny2022_withoutHistory", "ber-tiny42_withoutHistory"],
        True : ["bert-tiny42_withHistory", "bert-tiny2022_withHistory", "bert-tiny1337_withHistory"]
    }
}

In [ ]:
scores = {}

for model_name in MODEL_NAMES:
    for val in [True, False]:
        for model in models[model_name][val]:
            if model not in scores:
                print('Managing: ', model)
                model_ = CoQaModel(span_input_len=SPEX_LEN[model_name]["input_len"],
                                   s2s_input_len=SEQ2SEQ_LEN[model_name]["input_len"],
                                   s2s_output_len=SEQ2SEQ_LEN[model_name]["output_len"],
                                   device=DEVICE)
                model_.load_model(SAVING_PATH + model, model_name)
                scores[model] = model_evaluation(model_, qa_test)

                with open(SAVING_PATH + 'validation.obj', 'wb') as file:
                    pickle.dump(scores, file)
                model_.cpu()
                del model_

In [ ]:
with open(SAVING_PATH + 'validation.obj', 'rb') as file:
    scores = pickle.load(file)

In [ ]:
scores

{'distilroberta-base42_withoutHistory': 0.5880708681649026,
 'distilroberta-base2022_withoutHistory': 0.5839227024715794,
 'distilroberta-base1337_withoutHistory': 0.5853150134197084,
 'bert-tiny42_withoutHistory': 0.07786804880159426,
 'bert-tiny2022_withoutHistory': 0.07786804880159426,
 'distilroberta-base42_withHistory': 0.4571359443852196,
 'distilroberta-base2022_withHistory': 0.4597323655836258,
 'distilroberta-base1337_withHistory': 0.4597323655836258,
 'bert-tiny42_withHistory': 0.08071015581880442,
 'bert-tiny2022_withHistory': 0.09201652453340674,
 'bert-tiny1337_withHistory': 0.09201652453340674}

In [ ]:
pd.DataFrame(scores, index=["f1 score"], columns=scores.keys()).T

,f1 score
distilroberta-base42_withoutHistory,0.588071
distilroberta-base2022_withoutHistory,0.583923
distilroberta-base1337_withoutHistory,0.585315
bert-tiny42_withoutHistory,0.077868
bert-tiny2022_withoutHistory,0.077868
distilroberta-base42_withHistory,0.457136
distilroberta-base2022_withHistory,0.459732
distilroberta-base1337_withHistory,0.459732
bert-tiny42_withHistory,0.080710
bert-tiny2022_withHistory,0.092017


# Error Analysis

As last step for this assignment an error analysis is proposed. In the following section each best model is evaluated on each different sources of text from the test set. For each of them the 5 worst f1 scores with the respective question, answer and predicted answer are reported with a little analysis on the kind of answer for which the network shows the worst prediction.



In [ ]:
sources = qa_test.unique("source")          # get sources labels

In [ ]:
def evaluate(model, dataset, batch_size=10, history=False):
    
    iteration_steps = len(dataset) // batch_size

    batch_start = 0
    batch_end = batch_size

    predicted_answers = []
    
    print("°"*100)
    print("Predictiong Text")
    print("°"*100)
    
    progress_bar = tqdm(range(iteration_steps))
    while batch_end <= len(dataset):  
      logging.set_verbosity_error()
      predicted = (model.generate(dataset[batch_start : batch_end], history))
      logging.set_verbosity_info()
      for i in range(batch_size):
        predicted_answers.append({
            "id" : dataset[batch_start : batch_end]["id"][i], 
            "text" : predicted[i]
            })

      batch_start = batch_end 
      batch_end += batch_size

      progress_bar.update(1)

      if batch_end >= len(qa_test):
        batch_end = len(qa_test)

      if batch_start == batch_end:
        batch_end = len(qa_test) + 1

    f1_scores = {}
    mean_score = 0

    

    print("°"*100)
    print("Computing scores")
    print("°"*100)
    
    progress_bar = tqdm(range(len(predicted_answers)))
    for answer in predicted_answers:
      score = compute_f1(answer["text"], dataset.filter(lambda x: x["id"] == answer["id"])["answer"][0])
      mean_score += score

      score_dict = {
            "id" : answer["id"], 
            "score" : score,
            "prediction" : answer["text"]
            }

      if "5_worse" not in f1_scores.keys():
        f1_scores["5_worse"] = [score_dict]

      elif len(f1_scores["5_worse"]) < 5:
        f1_scores["5_worse"].append(score_dict)

      else:
        for min_score in f1_scores["5_worse"]:
          if score < min_score["score"]:
            f1_scores["5_worse"].remove(min_score)
            f1_scores["5_worse"].append(score_dict)
            break

      progress_bar.update(1)

    mean_score /= len(predicted_answers)
    f1_scores["mean_score"] = mean_score
    return f1_scores

In [ ]:
def display_eval_dict(eval_dict, dataset):

  data = []
  column_names = ["question", "correct_answer", "predicted_answer", "f1 score"]
  
  for example in eval_dict["5_worse"]:
    dataset_reference = dataset.filter(lambda x: x["id"] == example["id"])
    question = dataset_reference["question"][0]
    correct_answer = dataset_reference["answer"][0]
    row = [
        question,
        correct_answer,
        example["prediction"],
        example["score"]
    ]
    data.append(row)
 
  df = pd.DataFrame(data, columns=column_names)
  source = dataset["source"][0]
  print("°"*150)
  print(f"5 worst f1 scores for {source} source")
  print("°" * 150)
  print()
  print(tabulate(df, headers='keys', tablefmt='psql'))
  
  mean_f1 = eval_dict["mean_score"]
  print()
  print(f"mean f1 score: {mean_f1}")

## No history


### **DistilRoberta-base 42**

In [ ]:
model = CoQaModel(
    span_input_len=SPEX_LEN[MODEL_NAMES[0]]["input_len"],
    s2s_input_len=SEQ2SEQ_LEN[MODEL_NAMES[0]]["input_len"],
    s2s_output_len=SEQ2SEQ_LEN[MODEL_NAMES[0]]["output_len"],
    device=DEVICE
)

In [ ]:
model.load_model(SAVING_PATH + "Savings/distilroberta-base42_withoutHistory", MODEL_NAMES[0])

#### **mctest**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[0])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/141 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1410 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for mctest source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+------------------------------------------------------+------------------+--------------------+------------+
|    | question                                             | correct_answer   | predicted_answer   |   f1 score |
|----+------------------------------------------------------+------------------+--------------------+------------|
|  0 | Whose paint was it?                                  | the farmer       | Cotton             |          0 |
|  1 | Did they want Cotton to change the color of her fur? | no               | Yes                |          0 |
|  2 | who said that                                        | Asta.            | Shark

#### **race**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[1])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/164 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1640 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for race source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+--------------------------------+------------------+-------------------------+------------+
|    | question                       | correct_answer   | predicted_answer        |   f1 score |
|----+--------------------------------+------------------+-------------------------+------------|
|  0 | Is she carrying something?     | Yes              | a paper carrier bag     |          0 |
|  1 | Do I know her?                 | Yes              | no                      |          0 |
|  2 | How is she related to the boy? | mother           | Her grandmother         |          0 |
|  3 | What is in the bag?            | food             | a thermos with h

#### **cnn**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[2])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/163 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1630 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for cnn source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------+---------------------------+-----------------------------------+------------+
|    | question                         | correct_answer            | predicted_answer                  |   f1 score |
|----+----------------------------------+---------------------------+-----------------------------------+------------|
|  0 | What did he do?                  | Actor                     | he was a consultant               |          0 |
|  1 | Anything recent?                 | No                        | Yes                               |          0 |
|  2 | What happened in the early 80's? | Farina was cast in a film |

#### **wikipedia**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[3])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for wikipedia source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------------------+-------------------+-----------------------------------------------+------------+
|    | question                                     | correct_answer    | predicted_answer                              |   f1 score |
|----+----------------------------------------------+-------------------+-----------------------------------------------+------------|
|  0 | Was it founded the same year?                | Yes               | no                                            |          0 |
|  1 | Was he currently enrolled at the University? | He is not         | Yes                                         

#### **gutenberg**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[4])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for gutenberg source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+---------------------------------------------------------+--------------------------+----------------------------------------------------+------------+
|    | question                                                | correct_answer           | predicted_answer                                   |   f1 score |
|----+---------------------------------------------------------+--------------------------+----------------------------------------------------+------------|
|  0 | Who argued to Villa that he must have a name?           | Harley Kennan            | the president                                      |          0 |
|  1 | Who was the husband

### **Bertiny 42**

In [ ]:
model = CoQaModel(
    span_input_len=SPEX_LEN[MODEL_NAMES[1]]["input_len"],
    s2s_input_len=SEQ2SEQ_LEN[MODEL_NAMES[1]]["input_len"],
    s2s_output_len=SEQ2SEQ_LEN[MODEL_NAMES[1]]["output_len"],
    device=DEVICE
)

In [ ]:
model.load_model(SAVING_PATH + "Savings/bert-tiny42_withoutHistory", MODEL_NAMES[1])

#### **mctest**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[0])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/141 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1410 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for mctest source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------------------------------------------------+------------------------------+---------------------------------------------------+------------+
|    | question                                                                   | correct_answer               | predicted_answer                                  |   f1 score |
|----+----------------------------------------------------------------------------+------------------------------+---------------------------------------------------+------------|
|  0 | Who did she live with?                                                     | with her mommy and 5 sisters | a farm

#### **race**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[1])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/164 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1640 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for race source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+---------------------------------+------------------+--------------------------------------------------------+------------+
|    | question                        | correct_answer   | predicted_answer                                       |   f1 score |
|----+---------------------------------+------------------+--------------------------------------------------------+------------|
|  0 | Who is her daughter?            | Nicole           | natalie mauri a small dish a small dish dish, and a    |          0 |
|  1 | Where does Nicole live?         | Shanghai         | a dish dish soup a small salad soup a small salad      |          0 |
|  2 | How is

#### **cnn**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[2])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/163 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1630 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for cnn source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------+------------------+------------------------------------------------------------------------------+------------+
|    | question                         | correct_answer   | predicted_answer                                                             |   f1 score |
|----+----------------------------------+------------------+------------------------------------------------------------------------------+------------|
|  0 | Whom?                            | Dennis Farina    | michael bluster michael michael michael michael and a police michael michael |          0 |
|  1 | What did he do?                  | Actor     

#### **wikipedia**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[3])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for wikipedia source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+-------------------------------+------------------+--------------------------------------------------------------------------+------------+
|    | question                      | correct_answer   | predicted_answer                                                         |   f1 score |
|----+-------------------------------+------------------+--------------------------------------------------------------------------+------------|
|  0 | How many burroughs are there? | five             | two the north west the early 20th century the early 20th century         |          0 |
|  1 | in what city?                 | New York City    | the north west t

#### **gutenberg**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[4])
eval_dict = evaluate(model, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for gutenberg source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+-----------------------------------------------------+------------------+-----------------------------------------------------------+------------+
|    | question                                            | correct_answer   | predicted_answer                                          |   f1 score |
|----+-----------------------------------------------------+------------------+-----------------------------------------------------------+------------|
|  0 | What worked her way northward?                      | The _Ariel_      | a deep deep reef a deep reef a deep reef a deep reef      |          0 |
|  1 | What lay between the shore-reefs and ou

Generally the error analysis shows some difficulties od the network for discriminating the two words *yes* and *no*. The network (particularly **roberta-base** has learned when to summarize the entire answer into on of the two, but still makes mistakes discriminating them. Some of the errors are due to not well formatted answers, for example the answer to *"is she carrying something?"* is expected to be *"yes"* but the network answered *"a paper carrier bag"* which still is correct, but not formulated correctly. In most of the other cases there's general confusion in the answer, but the overall result could be considered pretty good.
Bertiny based model have much more difficulties in learning and answering and the general result shows the unsatisfing models.

## History


### **DistilRoberta-base 42**

In [ ]:
model = CoQaModel(
    span_input_len=SPEX_LEN[MODEL_NAMES[0]]["input_len"],
    s2s_input_len=SEQ2SEQ_LEN[MODEL_NAMES[0]]["input_len"],
    s2s_output_len=SEQ2SEQ_LEN[MODEL_NAMES[0]]["output_len"],
    device=DEVICE
)

In [ ]:
model.load_model(SAVING_PATH + "Savings/distilroberta-base42_withHistory", MODEL_NAMES[0])

#### **mctest**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[0])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/141 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1410 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for mctest source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------------------+------------------------------+--------------------+------------+
|    | question                                     | correct_answer               | predicted_answer   |   f1 score |
|----+----------------------------------------------+------------------------------+--------------------+------------|
|  0 | Who did she live with?                       | with her mommy and 5 sisters | Cotton             |          0 |
|  1 | who said that                                | Asta.                        | Asta's friend      |          0 |
|  2 | Did a little boy write the note              | No          

#### **race**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[1])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/164 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1640 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for race source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+--------------------------------+---------------------------------------------------------------------------------+------------------------------------+------------+
|    | question                       | correct_answer                                                                  | predicted_answer                   |   f1 score |
|----+--------------------------------+---------------------------------------------------------------------------------+------------------------------------+------------|
|  0 | What?                          | a paper carrier bag                                                             | the doorbell rings       

#### **cnn**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[2])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/163 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1630 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for cnn source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------+---------------------------+--------------------+------------+
|    | question                         | correct_answer            | predicted_answer   |   f1 score |
|----+----------------------------------+---------------------------+--------------------+------------|
|  0 | Is someone in showbiz?           | Yes.                      | Dennis Farina      |          0 |
|  1 | What did he do?                  | Actor                     | dapper             |          0 |
|  2 | Was he in movies?                | Yes                       | No                 |          0 |
|  3 | Anything recent?                 

#### **wikipedia**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[3])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for wikipedia source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+------------------------------------+---------------------------------------------------+--------------------+------------+
|    | question                           | correct_answer                                    | predicted_answer   |   f1 score |
|----+------------------------------------+---------------------------------------------------+--------------------+------------|
|  0 | How many burroughs are there?      | five                                              | Three              |          0 |
|  1 | in what city?                      | New York City                                     | Staten Island      |          0 |
|  2 | W

#### **gutenberg**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[4])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for gutenberg source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+-----------------------------------------------------+------------------+---------------------+------------+
|    | question                                            | correct_answer   | predicted_answer    |   f1 score |
|----+-----------------------------------------------------+------------------+---------------------+------------|
|  0 | What lay between the shore-reefs and outer-reefs?   | Lagoon           | the shore-reefs     |          0 |
|  1 | What coast did the Ariel work her way up leisurely? | Malaita          | Northward           |          0 |
|  2 | Who argued to Villa that he must have a name?       | Harley Kennan    | Hag

### **Bertiny 42**

In [ ]:
model = CoQaModel(
    span_input_len=SPEX_LEN[MODEL_NAMES[1]]["input_len"],
    s2s_input_len=SEQ2SEQ_LEN[MODEL_NAMES[1]]["input_len"],
    s2s_output_len=SEQ2SEQ_LEN[MODEL_NAMES[1]]["output_len"],
    device=DEVICE
)

In [ ]:
model.load_model(SAVING_PATH + "Savings/bert-tiny42_withHistory", MODEL_NAMES[1])

#### **mctest**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[0])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/141 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1410 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for mctest source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+------------------------------+------------------------------+-----------------------------+------------+
|    | question                     | correct_answer               | predicted_answer            |   f1 score |
|----+------------------------------+------------------------------+-----------------------------+------------|
|  0 | What color was Cotton?       | white                        | kitten kitten               |          0 |
|  1 | Where did she live?          | in a barn                    | kitten kitten kitten kitten |          0 |
|  2 | Did she live alone?          | no                           | yes kitten kitten           |   

#### **race**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[1])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/164 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1640 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for race source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+--------------------------------+---------------------+-----------------------------------------------------------------+------------+
|    | question                       | correct_answer      | predicted_answer                                                |   f1 score |
|----+--------------------------------+---------------------+-----------------------------------------------------------------+------------|
|  0 | What?                          | a paper carrier bag | grandmother grandmother grandmother                             |          0 |
|  1 | Who is her daughter?           | Nicole              | her daughter her mother her mother   

#### **cnn**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[2])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/163 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1630 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for cnn source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+----------------------------------+---------------------------+----------------------------------------+------------+
|    | question                         | correct_answer            | predicted_answer                       |   f1 score |
|----+----------------------------------+---------------------------+----------------------------------------+------------|
|  0 | Whom?                            | Dennis Farina             | professional                           |          0 |
|  1 | What did he do?                  | Actor                     | professional professional              |          0 |
|  2 | Is he still alive?               | No

#### **wikipedia**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[3])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for wikipedia source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+-------------------------------+------------------------------+--------------------+------------+
|    | question                      | correct_answer               | predicted_answer   |   f1 score |
|----+-------------------------------+------------------------------+--------------------+------------|
|  0 | How many burroughs are there? | five                         | two                |          0 |
|  1 | in what city?                 | New York City                | staten             |          0 |
|  2 | and state?                    | New York                     | staten             |          0 |
|  3 | Is staten island one?      

#### **gutenberg**

In [ ]:
test_on_source = qa_test.filter(lambda x: x["source"] == sources[4])
eval_dict = evaluate(model, test_on_source, history=True)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Predictiong Text
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/161 [00:00<?, ?it/s]

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
Computing scores
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°


  0%|          | 0/1610 [00:00<?, ?it/s]

In [ ]:
display_eval_dict(eval_dict, test_on_source)

°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°
5 worst f1 scores for gutenberg source
°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°°

+----+---------------------------------------------------+------------------+-------------------------------------------------+------------+
|    | question                                          | correct_answer   | predicted_answer                                |   f1 score |
|----+---------------------------------------------------+------------------+-------------------------------------------------+------------|
|  0 | What worked her way northward?                    | The _Ariel_      | coral reefs coral coral coral reefs coral coral |          0 |
|  1 | What lay between the shore-reefs and outer-reefs? | Lagoon           | reefs reefs reef

History field didn't seem to have any effect on the network, they got worst in this particular case. Probably the format of the history itself and the fact that it only appears on the fisrt chunk of the context made the task more difficult. A possible approach would be to insert the history field inside each chunk in which th context is divided to give the network always the field requested.